<pre>
This is our dataset that contains:-
    1. Timestamp,
    2. Your Age,
    3. Gender,
    4. State you reside in,
    5. City you reside in
    6. Product(s) you like/wish to purchase,
    7. Do you like to visit to parties more frequently.
    
In total we contain 210 rows and 7 columns out of which 6 are  of our use.
</pre>

In [1]:
import numpy as np
import pandas as pd

In [ ]:
df_raw_data = pd.read_csv('Customer Feedback (Responses) - Form responses 1 (1).csv')

df_raw_data.head()


### EDA
#### Droping the TimeStamp Column as it of no use for us

In [2]:
df_raw_data.drop(['Timestamp','Thing(s) on which you spend most of the time of your day'], axis = 1, inplace = True)

NameError: name 'df_raw_data' is not defined

### Renaming the columns

In [ ]:
df_raw_data.rename(columns = {'Your Age' : 'Age',
                              'State you reside in' : 'State',
                              'City you reside in' : 'City',
                              'Product(s) you like/wish to purchase' : 'Product_Purchase',
                              'Do you like to visit to parties more frequently?' : 'Party'}, inplace = True)

In [ ]:
df_raw_data.Product_Purchase.unique()

In [ ]:
df_raw_data.Product_Purchase.replace(to_replace = 'Headphones/Earphones ', value = 'Headphones/Earphones', inplace = True)
df_raw_data.Product_Purchase.replace(to_replace = 'Headphones/Earphone', value = 'Headphones/Earphones', inplace = True)
df_raw_data.Product_Purchase.replace(to_replace = ' Books', value = 'Books', inplace = True)
df_raw_data.Product_Purchase.replace(to_replace = ' Shoes', value = 'Shoes', inplace = True)
df_raw_data.Product_Purchase.replace(to_replace = ' Smartwatch', value = 'Smartwatch', inplace = True)
df_raw_data.Product_Purchase.replace(to_replace = 'Wallet, Shoes', value = 'Shoes', inplace = True)


In [ ]:
df_raw_data.head()

In [ ]:
# funtion to get the info about data .
def get_df_info(df, include_unique_values = False):
    col_name_list = list(df.columns)
    col_type_list = [type(df[col][0]) for col in col_name_list]
    col_null_count_list = [df[col].isnull().sum() for col in col_name_list]
    col_unique_count_list = [df[col].nunique() for col in col_name_list]
    col_memory_usage_list = [df[col].memory_usage(deep = True) for col in col_name_list]
    df_total_memory_usage = sum(col_memory_usage_list)/1048576
    if include_unique_values :
        col_unqiue_values_list = [df[col].unique() for col in col_name_list]
        df_info = pd.DataFrame({'column_name' : col_name_list, 'column_type': col_type_list,
                                'null_count': col_null_count_list, 'nunique':col_unique_count_list,
                                'unique_values' : col_unqiue_values_list
                               })
    else:
         df_info = pd.DataFrame({'column_name' : col_name_list, 'column_type': col_type_list,
                                'null_count': col_null_count_list, 'nunique':col_unique_count_list
                                }) 
    return df_info, df_total_memory_usage

    

In [ ]:
df_raw_data_info , df_raw_data_total_memory_usage = get_df_info(df_raw_data, True)
df_raw_data_info

### Data PreProcessing
#### Everything is in string So we will map into numeric using the cat.codes

In [ ]:
df_raw_data_copy = df_raw_data.copy()
df_raw_data_copy = df_raw_data_copy.astype('category')
df_raw_data_copy.head()

In [ ]:
df_raw_data_copy_info , df_raw_data_copy_total_memory_usage = get_df_info(df_raw_data_copy, True)
df_raw_data_copy_info

In [ ]:
for c in df_raw_data_copy.columns:
    df_raw_data_copy[c] = df_raw_data_copy[c].cat.codes
df_raw_data_copy.head()

In [ ]:
df_raw_data_copy_info , df_raw_data_copy_total_memory_usage = get_df_info(df_raw_data_copy, True)
df_raw_data_copy_info

<pre>
 Here, Product_Purchase is Y-variable or label
 And,  All others are X-variable or features
</pre>

### EDA 
#### Doing correlation on data set.

In [ ]:
df_raw_data_copy.describe()

<pre>
There are total of 200 entries(or rows).
Maximum People are of UttarPradesh(15) about 50 to 75%.
Maximum people belongs to Tier2 city(1).
Most of the people like to do party.
</pre>

## EDA through  data visualization.

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
sb.heatmap(df_raw_data_copy.corr(), cmap = 'coolwarm', annot = True, vmin = -1, vmax = 1)

#### The above is the correlation graph , through it we get to know that 
#### the highly correlated value with 'Product_Purchase' is city.

In [ ]:
sb.lmplot(data = df_raw_data_copy, x = 'Product_Purchase', y = 'Age')
plt.yticks(ticks = np.arange(0, 4), labels = ['0-17', '18-30', '30-45', '45 and above'])
plt.xticks(ticks = np.arange(0,7), labels = ['Headphones/Earphones','Books', 'Smartwatch', 'Shoes', 'OTT subscriptions', 'Perfume','Wallet'], rotation = 'vertical' )

In [ ]:
sb.lmplot(data = df_raw_data_copy, x = 'Product_Purchase', y = 'City')
plt.yticks(ticks = np.arange(0, 3), labels = ['Tier1', 'Tier2', 'Tier3'])
plt.xticks(ticks = np.arange(0,7), labels = ['Headphones/Earphones','Books', 'Smartwatch', 'Shoes', 'OTT subscriptions', 'Perfume','Wallet'], rotation = 'vertical' )

In [ ]:
sb.lmplot(data = df_raw_data_copy, x = 'Product_Purchase', y = 'Party')
plt.yticks(ticks = np.arange(0, 2), labels = ['No', 'Yes'])
plt.xticks(ticks = np.arange(0,7), labels = ['Headphones/Earphones','Books', 'Smartwatch', 'Shoes', 'OTT subscriptions', 'Perfume','Wallet'], rotation = 'vertical' )

### Creating machine learning models.
##### We are using r2_score and mean_absolute_error metrices to evaluate our model.

In [ ]:
from sklearn.model_selection import cross_validate, train_test_split

from sklearn.linear_model import LinearRegression as LR 

from sklearn.ensemble import RandomForestRegressor as RFR

from sklearn.svm import SVC

from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
def show_model_eval_table(model_attrib):
    df_model_eval = pd.DataFrame({'model': model_attrib['model_names'],
                                  'feature_count': model_attrib['model_feature_counts'], 
                                  'feature_names': model_attrib['model_feature_names'], 
                                  'r2': model_attrib['model_r2_scores'], 
                                  'mae': model_attrib['model_mae_scores']})
    return df_model_eval.round(2)

In [ ]:
model_attrib = {
    'model_names': [],
    'model_feature_counts': [],
    'model_feature_names': [],
    'model_r2_scores': [],
    'model_mae_scores': []
}

### Creating model 1 with all features.

In [ ]:
X_data = df_raw_data_copy.loc[:, ['Age', 'Gender', 'State', 'City', 'Party'] ]

X_data.shape

In [ ]:
X_train, X_test, Y_train, Y_test= train_test_split(X_data, df_raw_data_copy.Product_Purchase, random_state = 0)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

In [ ]:
### Linear Regression model
lr_model_1 = LR()
lr_model_1.fit(X_train, Y_train)
y_hat_lr_model_1 = lr_model_1.predict(X_test)
model_attrib['model_names'].append('lr_model_1')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_lr_model_1))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_lr_model_1))

#RandomForestRegressor
rf_model_1 = RFR(random_state = 0)
rf_model_1.fit(X_train, Y_train)
y_hat_rf_model_1 = rf_model_1.predict(X_test)
model_attrib['model_names'].append('rf_model_1')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_rf_model_1))
model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_rf_model_1))


#SVC
sv_model_1 = SVC(random_state = 0)
sv_model_1.fit(X_train, Y_train)
y_hat_sv_model_1 = sv_model_1.predict(X_test)
model_attrib['model_names'].append('sv_model_1')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_sv_model_1))
model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_sv_model_1))




In [ ]:
#SGDClassifier
from  sklearn.linear_model import SGDClassifier as SDG
sg_model_1 = SDG(random_state = 0)
sg_model_1.fit(X_train, Y_train)
y_hat_sg_model_1 = sg_model_1.predict(X_test)
model_attrib['model_names'].append('sg_model_1')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_sg_model_1))
model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_sg_model_1))




In [ ]:
show_model_eval_table(model_attrib)

### Creating features set 2 by dropping 'Age' Column.


In [ ]:
X_data2 = X_data.drop(['Age'], axis  = 1)
X_data2.shape

In [ ]:
X_train2, X_test2 = X_train.drop(['Age'], axis  = 1), X_test.drop(['Age'], axis  = 1)
print(X_train2.shape, X_test2.shape)

In [ ]:
### Linear Regression model
lr_model_2 = LR()
lr_model_2.fit(X_train2, Y_train)
y_hat_lr_model_2 = lr_model_2.predict(X_test2)
model_attrib['model_names'].append('lr_model_2')
model_attrib['model_feature_counts'].append(X_train2.shape[1])
model_attrib['model_feature_names'].append(list(X_train2.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_lr_model_2))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_lr_model_2))

#RandomForestRegressor
rf_model_2 = RFR(random_state = 0)
rf_model_2.fit(X_train2, Y_train)
y_hat_rf_model_2 = rf_model_2.predict(X_test2)
model_attrib['model_names'].append('rf_model_2')
model_attrib['model_feature_counts'].append(X_train2.shape[1])
model_attrib['model_feature_names'].append(list(X_train2.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_rf_model_2))
model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_rf_model_2))



#SVC
sv_model_2 = SVC(random_state = 0)
sv_model_2.fit(X_train2, Y_train)
y_hat_sv_model_2 = sv_model_2.predict(X_test2)
model_attrib['model_names'].append('sv_model_2')
model_attrib['model_feature_counts'].append(X_train2.shape[1])
model_attrib['model_feature_names'].append(list(X_train2.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_sv_model_2))
model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_sv_model_2))


#SGDClassifier
sg_model_2 = SDG(random_state = 0)
sg_model_2.fit(X_train2, Y_train)
y_hat_sg_model_2 = sg_model_2.predict(X_test2)
model_attrib['model_names'].append('sg_model_2')
model_attrib['model_feature_counts'].append(X_train2.shape[1])
model_attrib['model_feature_names'].append(list(X_train2.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_sg_model_2))
model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_sg_model_2))



In [ ]:
show_model_eval_table(model_attrib)

### Creating the LR model3 by manual tuning the hyperparameters.

In [ ]:
### Linear Regression model
lr_model_3 = LR(normalize = True)
lr_model_3.fit(X_train, Y_train)
y_hat_lr_model_3 = lr_model_3.predict(X_test)
model_attrib['model_names'].append('lr_model_3')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_lr_model_3))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_lr_model_3))

In [ ]:
show_model_eval_table(model_attrib)

### Creating the LR model4 using manual hyperparameter tuning features set 2.

In [ ]:
### Linear Regression model
lr_model_4 = LR(normalize = True)
lr_model_4.fit(X_train2, Y_train)
y_hat_lr_model_4 = lr_model_4.predict(X_test2)
model_attrib['model_names'].append('lr_model_4')
model_attrib['model_feature_counts'].append(X_train2.shape[1])
model_attrib['model_feature_names'].append(list(X_train2.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_lr_model_4))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_lr_model_4))

In [ ]:
show_model_eval_table(model_attrib)

In [ ]:
### Using Lasso and ElasticNet algo to make our model.

In [ ]:
from sklearn.linear_model import Lasso as LS
ls_model_1 = LS(random_state = 0)
ls_model_1.fit(X_train, Y_train)
y_hat_ls_model_1 = ls_model_1.predict(X_test)
model_attrib['model_names'].append('ls_model_1')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_ls_model_1))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_ls_model_1))

In [ ]:
show_model_eval_table(model_attrib)

In [ ]:
from sklearn.linear_model import ElasticNet as EN
en_model_1 = EN(random_state = 0)
en_model_1.fit(X_train, Y_train)
y_hat_en_model_1 = en_model_1.predict(X_test)
model_attrib['model_names'].append('en_model_1')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_en_model_1))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_en_model_1))

In [ ]:
show_model_eval_table(model_attrib)

In [ ]:
### Creating the models using automatic Hyperparameter Tuning using GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV


In [ ]:
hp_grid = {'max_iter' : [100, 200, 500, 1000, 2000, 5000],
                              'warm_start':[True, False],
             'selection': ['random', 'cyclic']}

gscv_en = GridSearchCV(EN(random_state = 0), param_grid =hp_grid, n_jobs = 5, cv = 5, verbose = 10 )
gscv_en.fit(X_data, df_raw_data_copy.Product_Purchase)

In [ ]:
gscv_en.best_params_

In [ ]:
en_model_2 = EN(random_state = 0, max_iter = 100, selection = 'random', warm_start = True)
en_model_2.fit(X_train, Y_train)
y_hat_en_model_2 = en_model_2.predict(X_test)
model_attrib['model_names'].append('en_model_2')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_en_model_2))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_en_model_2))

In [ ]:
show_model_eval_table(model_attrib)

In [ ]:
gscv_en = GridSearchCV(LS(random_state = 0), param_grid =hp_grid, n_jobs = 5, cv = 5, verbose = 10 )
gscv_en.fit(X_data, df_raw_data_copy.Product_Purchase)

In [ ]:
gscv_en.best_params_

In [ ]:
ls_model_2 = LS(random_state = 0, max_iter = 100, selection = 'random', warm_start = True)
ls_model_2.fit(X_train, Y_train)
y_hat_ls_model_2 = ls_model_2.predict(X_test)
model_attrib['model_names'].append('ls_model_2')
model_attrib['model_feature_counts'].append(X_train.shape[1])
model_attrib['model_feature_names'].append(list(X_train.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_ls_model_2))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_ls_model_2))

In [ ]:
show_model_eval_table(model_attrib)

### Lasso and ElasticNet mdoel using the automated Hyperparamter Tuning and feature set2. 

In [ ]:
ls_model_3 = LS(random_state = 0, max_iter = 100, selection = 'random', warm_start = True)
ls_model_3.fit(X_train2, Y_train)
y_hat_ls_model_3 = ls_model_3.predict(X_test2)
model_attrib['model_names'].append('ls_model_3')
model_attrib['model_feature_counts'].append(X_train2.shape[1])
model_attrib['model_feature_names'].append(list(X_train2.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_ls_model_3))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_ls_model_3))

In [ ]:
en_model_3 = EN(random_state = 0, max_iter = 100, selection = 'random', warm_start = True)
en_model_3.fit(X_train2, Y_train)
y_hat_en_model_3 = en_model_3.predict(X_test2)
model_attrib['model_names'].append('en_model_3')
model_attrib['model_feature_counts'].append(X_train2.shape[1])
model_attrib['model_feature_names'].append(list(X_train2.columns))
model_attrib['model_r2_scores'].append(r2_score(Y_test, y_hat_en_model_3))

model_attrib['model_mae_scores'].append(mean_absolute_error(Y_test, y_hat_en_model_3))

In [ ]:
show_model_eval_table(model_attrib)

<pre>
According to our above table the most suitable model is en_model_2 we can also take ls_model_2, 
the one with automated Hyperparameter Tuning.
</pre>